In [60]:
import pandas as pd
import re
fpkm_df = pd.read_csv("/media/kusterlab/internal_projects/active/TOPAS/WP31/FPKM/fpkms_230725mitINFORM.csv")
sample_annotation_df = pd.read_csv("/media/kusterlab/internal_projects/active/TOPAS/WP31/Searches/patient_annotation_231023_PAN_cancer_copy.csv")
fpkm_df.set_index('Unnamed: 0',inplace=True)
fpkm_df_ids = [x.replace(' Z-score','') for x in fpkm_df.columns]
sample_annotation_df_ids = sample_annotation_df['Sample name'].tolist()
def remove_replicate_from_ids(list_to_proceed):
    import re
    regex = re.compile(r"-R[0-9]$", re.IGNORECASE)
    list_regex= [regex.findall(x) for x in list_to_proceed]
    list_regex = [x[0] if len(x)>0 else '' for x in list_regex]
    for i,x in enumerate(list_regex):
        list_to_proceed[i] = list_to_proceed[i].replace(x,'')
    return list_to_proceed

fpkm_df_ids = remove_replicate_from_ids(fpkm_df_ids)
sample_annotation_df_ids = remove_replicate_from_ids(sample_annotation_df_ids)

patient_ids_to_be_requested = pd.DataFrame(list(set([x for x in sample_annotation_df_ids if x not in fpkm_df_ids])),columns=['ids'])


sample_annotation_df['Sample_name_rep_truncated'] = sample_annotation_df['Sample name'].str.replace(r'-R[0-9]$', '', regex=True)
final = patient_ids_to_be_requested.merge(sample_annotation_df,right_on='Sample_name_rep_truncated',left_on='ids',how='inner')
final = final[['ids','Batch Name']]
final = final.drop_duplicates(subset=['ids'])
final.to_excel('/home/amir/Desktop/fpkm_ids_tobe_requested.xlsx')